In [53]:
import pandas as pd
import numpy as np
import nltk

np.random.seed(543)

dictionary = {}

df = pd.read_csv('noticias_estadao.csv')

for index, row in df.iterrows():
    tokens = [word.lower() for word in row.titulo.split(' ')] 
    print (tokens)

['pt', 'espera', '30', 'mil', 'pessoas', 'em', 'festa', 'na', 'esplanada']
['alckmin', 'toma', 'posse', 'de', 'olho', 'no', 'planalto']
['seis', 'obstáculos', 'e', 'desafios', 'do', 'segundo', 'mandato', 'de', 'dilma']
['veja', 'os', 'desafios', 'dos', 'governadores', 'que', 'assumem', 'nesta', 'quinta']
['pt', 'impulsiona', 'cerimônia', 'de', 'posse', 'da', 'dilma', 'nas', 'redes', 'sociais']
['dilma', 'veta', 'projeto', 'de', 'suplicy', 'que', 'cria', 'linha', 'oficial', 'da', 'pobreza']
['dilma', 'anuncia', '14', 'ministros', 'e', 'conclui', 'reforma', 'ministerial']
['dilma', 'anuncia', 'últimos', '14', 'ministros', 'e', 'fecha', 'equipe']
['ex-governador', 'critica', 'alckmin', 'pela', 'falta', 'de', 'água', 'em', 'sp']
['veja', 'lista', 'completa', 'de', 'ministros', 'anunciados', 'por', 'dilma']
["'pt", 'não', 'pode', 'se', "queixar',", 'afirma', 'futuro', 'articulador', '', '']
['planalto', 'confirma', 'juca', 'ferreira', 'no', 'ministério', 'da', 'cultura']
['pimentel', 'divul

['pizzolato', 'ficará', 'livre', 'por', 'um', 'ano', 'antes', 'de', 'caso', 'voltar', 'a', 'ser', 'julgado', 'na', 'itália']
['entenda', 'a', 'diferença', 'entre', 'plebiscito', 'e', 'referendo']
['após', 'derrotas,', 'psb', 'fará', '‘oposição', 'de', 'esquerda’']
['supremo', 'em', 'pauta:', 'o', 'não', 'estado', 'de', 'direito', '']
['empresário', 'diz', 'que', 'doleiro', 'pagou', 'dívida', 'em', 'nome', 'da', 'petrobrás']
['fazenda', 'terá', 'novo', 'nome', 'antes', 'do', 'g-20', '', '']
['partidos', 'travam', 'guerra', 'surda', 'por', 'espaços', 'na', 'esplanada', '', '']
['não', 'tem', 'refresco', '', '']
['câmara', 'derruba', 'decreto', 'de', 'conselho', 'popular', 'de', 'dilma']
['‘não', 'existe', 'canetada', 'que', 'acabe', 'com', 'o', 'tal', 'é', 'dando', 'que', 'se', 'recebe’']
['presidente', 'recua', 'e', 'já', 'admite', 'abrir', 'mão', 'de', 'plebiscito']
['repressão', 'e', 'resistência:', 'livro', 'analisa', 'relações', 'entres', 'gays', 'e', 'ditadura']
['dilma:', 'união',

['luciana', 'genro', 'diz', 'que', 'seu', 'programa', 'é', "'de", "transição'"]
['dilma', 'vai', 'para', 'rua', 'contra', 'imagem', 'de', "'encastelada'"]
['representante', 'de', 'aécio', 'debate', 'direitos', 'dos', 'deficientes']
['para', 'genro,', 'programa', 'do', 'psol', 'não', 'é', 'socialista,', 'mas', 'de', 'transição']
['aécio', 'diz', 'que', 'vai', 'resgatar', 'a', 'confiança', 'no', 'brasil']
['skaf', 'faz', 'campanha', 'solo', 'em', 'cidades', 'com', 'pmdb', 'rachado']
['armínio', 'diz', 'que', 'só', 'vai', 'para', 'o', 'governo', 'com', 'aécio']
['skaf', 'faz', 'campanha', 'solo', 'em', 'cidades', 'com', 'pmdb', 'rachado']
['dirigentes', 'acham', 'difícil', 'tirar', 'amaral', 'da', 'direção', 'do', 'psb']
['janot', 'é', 'contra', 'conceder', 'direito', 'de', 'resposta', 'a', 'dilma']
['na', 'tv,', 'aécio', 'fala', 'em', "'ampla", "discussão'", 'do', 'fator', 'previdenciário']
['queda', 'de', 'marina', 'não', 'altera', 'dificuldades', 'do', 'governo']
['dilma', 'diz', 'que'

['procuradoria-geral', 'eleitoral', 'pede', 'multa', 'contra', 'aloysio', 'nunes']
['luciana', 'genro', 'grava', 'programas', 'para', 'campanha', 'presidencial']
['coordenador', 'do', 'pt', 'se', 'diz', 'tranquilo', 'para', 'rebater', 'aécio']
['marina', 'vê', 'indícios', 'de', "'ilicitudes'", 'em', 'aeroporto', 'em', 'mg']
['dilma', 'vai', 'à', 'rua,', 'marca', 'três', 'eventos', 'públicos', 'e', 'reage', 'a', "'aezão'", 'no', 'rio']
['dilma', 'vai', 'às', 'ruas', 'e', 'lança', 'ofensiva', 'contra', "'chapa", "aezão'"]
['campos', 'diz', 'que', 'não', 'vê', 'incoerência', 'em', 'apoiar', 'alckmin']
['psb', 'sempre', 'defendeu', 'candidato', 'próprio', 'em', 'mg,', 'diz', 'marina']
['lula:', 'seria', 'irresponsabilidade', 'pensar', 'em', 'voltar', 'em', '2018']
['lula', 'diz', 'que', '2º', 'turno', 'permite', "'construir", 'a', "coalizão'"]
['aliança', 'de', 'contrários']
["'reduzimos", 'a', 'inflação', 'deixada', 'por', "fhc',", 'diz', 'lula']
['para', 'lula,', 'a', 'política', 'está',

['líder', 'do', 'dem', 'pedirá', 'convocação', 'de', 'mantega', 'e', 'adams', '']
['dilma', 'aprovou', 'refinaria', 'no', 'japão', 'ciente', 'de', 'cláusula']
["'ele", 'sabe', 'ficar', 'perto', 'dos', "políticos',", 'diz', 'jornalista', '', '']
['dilma', 'aprovou', 'refinaria', 'no', 'japão', 'ciente', 'de', 'cláusula', '', '']
['perplexo,', 'pt', 'ataca', 'oposição', 'e', 'defende', 'estatal']
['petrobrás', 'demite', 'autor', 'de', 'parecer', 'que', 'dilma', 'chamou', 'de', '‘falho’', 'e', '‘incompleto’']
['torturadores', 'arrancavam', 'arcadas', 'dentárias', 'e', 'cortavam', 'dedos,', 'para', 'impedir', 'identificação', 'dos', 'mortos']
['pt', 'adota', 'como', 'estratégia', 'concentrar', 'crise', 'na', 'petrobras']
['supremo', 'decide', 'na', 'próxima', 'semana', 'se', 'julga', 'azeredo']
['stf', 'decide', 'quinta', 'se', 'julga', 'azeredo', 'por', 'mensalão', 'tucano']
['é', "'inacreditável'", 'dilma', 'ter', 'aprovado', 'compra', 'de', 'pasadena,', 'diz', 'marina']
["'minha", 'únic